<a href="https://colab.research.google.com/github/AdithiyaG/Langchain/blob/main/Langchain/DLAI/Functions%2CTools%2CAgents_with_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Functions,Tools,Agents with Langchain by Harison Chase
Deeplearing.ai short course


> 30 April 2024



# Setup

In [ ]:
!pip -q install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00


In [ ]:
from openai import AzureOpenAI

In [ ]:
import os
from google.colab import userdata
os.environ["AZURE_OPENAI_API_KEY"]=userdata.get('OPENAI_API_KEY')
os.environ["AZURE_OPENAI_ENDPOINT"]=userdata.get('OPENAI_ENDPOINT')
os.environ["OPENAI_API_VERSION"]="2024-02-15-preview"

client = AzureOpenAI()

In [ ]:
!pip install -q langchain langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 8.8 MB/s eta 0:00:00


In [ ]:
from langchain_openai  import AzureChatOpenAI
model=AzureChatOpenAI(model="gpt-4-turbo")
model_gpt_4=AzureChatOpenAI(model="gpt-4-32k")

# Lesson 1

In [ ]:
import json

def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [ ]:
get_current_weather("SF")

'{"location": "SF", "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

OpenAI function format


In [ ]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

In [ ]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

In [ ]:
response=client.chat.completions.create(
    model="gpt-4-32k",
    messages=messages,
    functions=functions,

)

In [ ]:
response

ChatCompletion(id='chatcmpl-9JcINfUAabfdZMKReFELhQ1KaWb3m', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather'), tool_calls=None), content_filter_results={})], created=1714462467, model='gpt-4-32k', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=18, prompt_tokens=82, total_tokens=100), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}])

 # Lesson 3


## Pydantic
Pydantic is data validation library for python

* works with python type annotations.but rather than static type checking,they are actively used at runtime for dta validation

* Provides built in methods sserialize/deserialize methods to/from JSON,dict etc

* Langchain leverages Pydantic to create JSON scheme describing function

In [ ]:
from typing import List
from pydantic import BaseModel,Field

In [ ]:
class User:
  def __init__(self,name:str,age:int,email:str):
    self.name=name
    self.age=age
    self.email=email

In [ ]:
foo=User("Joe",32,"abc@x.com")

In [ ]:
foo.age

32

In [ ]:
foo=User("Joe","NA","abc@x.com")

In [ ]:
foo.age

'NA'

In [ ]:
class pUser(BaseModel):
  name:str
  age:int
  email:str

In [ ]:
foo=pUser(name="Joe",age="NA",email="abc@x.com")

ValidationError: 1 validation error for pUser
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='NA', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/int_parsing

## Pydantic to OpenAI functions

Function description is necessary, Field Description is optional

In [ ]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")

In [ ]:
from langchain_core.utils.function_calling import convert_to_openai_function

convert_to_openai_function fails to capture Field description cmp to convert_pydantic_to_openai_functions(deprecrated)

In [ ]:
weather_function

{'name': 'WeatherSearch',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'type': 'object',
  'properties': {'airport_code': {'type': 'string'}},
  'required': ['airport_code']}}

In [ ]:
from langchain_openai  import AzureChatOpenAI

In [ ]:
model=AzureChatOpenAI(model="gpt-4-32k")

In [ ]:
model.invoke("what is the weather in SF today?", functions=[weather_function])

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "airport_code": "SFO"\n}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 62, 'total_tokens': 80}, 'model_name': 'gpt-4-32k', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'function_call', 'logprobs': None, 'content_filter_results': {}}, id='run-1617ec15-3c19-4c02-a284-02bc15029fd9-0')

In [ ]:
model_with_functions=model.bind(functions=[weather_function])
model_with_functions.invoke("what is the weather in SF today?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "airport_code": "SFO"\n}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 62, 'total_tokens': 80}, 'model_name': 'gpt-4-32k', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'function_call', 'logprobs': None, 'content_filter_results': {}}, id='run-ed27a676-46f7-42a3-aa92-188d07cd4660-0')

In [ ]:
model_with_functions.invoke("hi!")

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 56, 'total_tokens': 66}, 'model_name': 'gpt-4-32k', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-1de05419-3020-4fb3-83cc-54e941ef18fa-0')

### Forcing Function Call

In [ ]:
model_with_forced_function = model.bind(functions=[weather_function], function_call={"name":"WeatherSearch"})

In [ ]:
model_with_forced_function.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "airport_code": "SFO"\n}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 68, 'total_tokens': 79}, 'model_name': 'gpt-4-32k', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, id='run-18f314be-470f-44e4-8a7d-0b7c2360ff41-0')

In [ ]:
model_with_forced_function.invoke("hi!")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "airport_code": "JFK"\n}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 63, 'total_tokens': 74}, 'model_name': 'gpt-4-32k', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, id='run-71d5a831-26e7-4063-80fb-278e26aa5ce1-0')

# Lesson 4 Tagging and Extraction

### Tagging

In [ ]:
# Tagging and Extraction Using OpenAI functions

from typing import List
from pydantic import BaseModel, Field
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

class Tagging(BaseModel):
    """Tag the piece of text with particular info."""
    sentiment: str = Field(description="sentiment of text, should be `pos`, `neg`, or `neutral`")
    language: str = Field(description="language of text (should be ISO 639-1 code)")


In [ ]:
convert_pydantic_to_openai_function(Tagging)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.2.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


{'name': 'Tagging',
 'description': 'Tag the piece of text with particular info.',
 'parameters': {'properties': {'sentiment': {'description': 'sentiment of text, should be `pos`, `neg`, or `neutral`',
    'type': 'string'},
   'language': {'description': 'language of text (should be ISO 639-1 code)',
    'type': 'string'}},
  'required': ['sentiment', 'language'],
  'type': 'object'}}

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI



tagging_functions = [convert_pydantic_to_openai_function(Tagging)]

prompt = ChatPromptTemplate.from_messages([
    ("system", "Think carefully, and then tag the text as instructed"),
    ("user", "{input}")
])

model_with_functions = model.bind(
    functions=tagging_functions,
    function_call={"name": "Tagging"}
)

tagging_chain = prompt | model_with_functions



In [ ]:
tagging_chain.invoke({"input": "I love langchain"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n"sentiment": "pos",\n"language": "en"\n}', 'name': 'Tagging'}}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 105, 'total_tokens': 120}, 'model_name': 'gpt-4-32k', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, id='run-4c2d712c-75af-4346-aa25-b7af2bef1511-0')

In [ ]:
tagging_chain.invoke({"input": "non mi piace questo cibo"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n"sentiment": "neg",\n"language": "it"\n}', 'name': 'Tagging'}}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 108, 'total_tokens': 123}, 'model_name': 'gpt-4-32k', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, id='run-aec59328-db3c-4b43-9c82-189dae790bb4-0')

In [ ]:
JsonOutputFunctionsParser

langchain_core.output_parsers.openai_functions.JsonOutputFunctionsParser

In [ ]:

from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

tagging_chain = prompt | model_with_functions | JsonOutputFunctionsParser()

tagging_chain.invoke({"input": "non mi piace questo cibo"})

{'sentiment': 'neg', 'language': 'it'}

### Extraction
Extraction is similar to tagging, but used for extracting multiple pieces of information.

In [ ]:
from typing import Optional
class Person(BaseModel):
    """Information about a person."""
    name: str = Field(description="person's name")
    age: Optional[int] = Field(description="person's age")

class Information(BaseModel):
    """Information to extract."""
    people: List[Person] = Field(description="List of info about people")

In [ ]:
convert_pydantic_to_openai_function(Information)

{'name': 'Information',
 'description': 'Information to extract.',
 'parameters': {'$defs': {'Person': {'description': 'Information about a person.',
    'properties': {'name': {'description': "person's name", 'type': 'string'},
     'age': {'anyOf': [{'type': 'integer'}, {'type': 'null'}],
      'description': "person's age"}},
    'required': ['name', 'age'],
    'type': 'object'}},
  'properties': {'people': {'description': 'List of info about people',
    'items': {'description': 'Information about a person.',
     'properties': {'name': {'description': "person's name", 'type': 'string'},
      'age': {'anyOf': [{'type': 'integer'}, {'type': 'null'}],
       'description': "person's age"}},
     'required': ['name', 'age'],
     'type': 'object'},
    'type': 'array'}},
  'required': ['people'],
  'type': 'object'}}

In [ ]:
extraction_functions = [convert_pydantic_to_openai_function(Information)]
extraction_model = model.bind(functions=extraction_functions, function_call={"name": "Information"})

In [ ]:
extraction_model.invoke("Joe is 30, his mom is Martha")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "people": [\n    {\n      "name": "Joe",\n      "age": 30\n    },\n    {\n      "name": "Martha",\n      "age": null\n    }\n  ]\n}', 'name': 'Information'}}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 76, 'total_tokens': 121}, 'model_name': 'gpt-4-32k', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, id='run-6f3de8ec-e0f4-4811-bc16-febdf5347540-0')

In [ ]:

prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract the relevant information, if not explicitly provided do not guess. Extract partial info"),
    ("human", "{input}")
])

extraction_chain = prompt | extraction_model

In [ ]:
extraction_chain.invoke({"input": "Joe is 30, his mom is Martha"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "people": [\n    {\n      "name": "Joe",\n      "age": 30\n    },\n    {\n      "name": "Martha",\n      "age": null\n    }\n  ]\n}', 'name': 'Information'}}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 93, 'total_tokens': 138}, 'model_name': 'gpt-4-32k', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, id='run-85a05e06-0d42-4f93-9a1c-ebb5f961561e-0')

In [ ]:
extraction_chain = prompt | extraction_model | JsonOutputFunctionsParser()

extraction_chain.invoke({"input": "Joe is 30, his mom is Martha"})

{'people': [{'name': 'Joe', 'age': 30}, {'name': 'Martha', 'age': None}]}

In [ ]:

from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="people")

extraction_chain.invoke({"input": "Joe is 30, his mom is Martha"})

[{'name': 'Joe', 'age': 30}, {'name': 'Martha', 'age': None}]

In [ ]:
extraction_chain.invoke({"input": """Park Jimin, a sprightly 25-year-old with a passion for dance, greeted the morning at his bakery. Across the street, Dr. Amelia Hernandez, a seasoned physician at 62, reviewed patient charts. Young Maya Lopez, 8, skipped down the sidewalk, clutching a library book. Her grandfather, Mr. Chen, a retired carpenter with kind eyes (his age a delightful mystery), sat on a park bench, feeding pigeons.
At the bustling cafe, Sarah Jones, a lawyer with a fierce intellect, sipped coffee with her friend, David Lee, a 30-year-old software developer. In the distance, a group of energetic teenagers, led by the ever-enthusiastic 17-year-old Gabriela Rodriguez, practiced for a school play.
Ms. Evelyn Wright, a librarian with a wealth of knowledge (her age truly irrelevant), meticulously shelved books. Meanwhile, Mr. Tanaka, a 78-year-old with a youthful spirit, tended his immaculate flower garden.
The city buzzed with life. Delivery person, Omar Khan, 22 and always reliable, wove through the streets. Ms. Garcia, a community leader with boundless energy (though some whispered her age might be closer to 80 than 70), organized a neighborhood cleanup.
In the afternoon, young firefighter, Daniel Kim, 28 and brave, trained with his team. Nearby, Mrs. Patel, a tailor with a meticulous eye (age a mere number), meticulously crafted a beautiful sari.
As the sun dipped below the horizon, young entrepreneur, Anika Sharma, 19 and full of ideas, brainstormed new business ventures. Across town, Mr. and Mrs. Thompson, a loving couple whose age didn't matter as they held hands, strolled through the park.
This city, a tapestry woven from threads of countless lives, young and old, each with a story, a purpose, and a heart that beat to the rhythm of the bustling urban jungle."""})

[{'name': 'Park Jimin', 'age': 25},
 {'name': 'Dr. Amelia Hernandez', 'age': 62},
 {'name': 'Maya Lopez', 'age': 8},
 {'name': 'Mr. Chen', 'age': None},
 {'name': 'Sarah Jones', 'age': None},
 {'name': 'David Lee', 'age': 30},
 {'name': 'Gabriela Rodriguez', 'age': 17},
 {'name': 'Ms. Evelyn Wright', 'age': None},
 {'name': 'Mr. Tanaka', 'age': 78},
 {'name': 'Omar Khan', 'age': 22},
 {'name': 'Ms. Garcia', 'age': None},
 {'name': 'Daniel Kim', 'age': 28},
 {'name': 'Mrs. Patel', 'age': None},
 {'name': 'Anika Sharma', 'age': 19},
 {'name': 'Mr. Thompson', 'age': None},
 {'name': 'Mrs. Thompson', 'age': None}]


## Doing it for real

We can apply tagging to a larger body of text.

For example, let's load this blog post and extract tag information from a sub-set of the text.

In [ ]:
#Tagging
from langchain.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
documents = loader.load()

doc = documents[0]

page_content = doc.page_content[:10000]



class Overview(BaseModel):
    """Overview of a section of text."""
    summary: str = Field(description="Provide a concise summary of the content.")
    language: str = Field(description="Provide the language that the content is written in.")
    keywords: str = Field(description="Provide keywords related to the content.")

overview_tagging_function = [
    convert_pydantic_to_openai_function(Overview)
]
tagging_model = model.bind(
    functions=overview_tagging_function,
    function_call={"name":"Overview"}
)
tagging_chain = prompt | tagging_model | JsonOutputFunctionsParser()

tagging_chain.invoke({"input": page_content})


{'summary': "The blog post discusses the concept of building autonomous agents powered by Large Language Models (LLMs). It covers three main components of such a system: Planning, Memory, and Tool Use. In planning, the agent breaks down large tasks into smaller, manageable subgoals and performs self-reflection to improve. The memory component discusses the use of short-term and long-term memory in LLMs. The use of tools refers to the agent's ability to call external APIs for additional information. The post also discusses various case studies and challenges.",
 'language': 'English',
 'keywords': 'Large Language Models, Autonomous Agents, Planning, Memory, Tool Use, Self-Reflection, Task Decomposition'}

In [ ]:
#Extraction
class Paper(BaseModel):
    """Information about papers mentioned."""
    title: str
    author: Optional[str]


class Info(BaseModel):
    """Information to extract"""
    papers: List[Paper]

paper_extraction_function = [
    convert_pydantic_to_openai_function(Info)
]
extraction_model = model.bind(
    functions=paper_extraction_function,
    function_call={"name":"Info"}
)
extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="papers")

extraction_chain.invoke({"input": page_content})

template = """A article will be passed to you. Extract from it all papers that are mentioned by this article.

Do not extract the name of the article itself. If no papers are mentioned that's fine - you don't need to extract any! Just return an empty list.

Do not make up or guess ANY extra information. Only extract what exactly is in the text."""

prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", "{input}")
])

extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="papers")

In [ ]:
extraction_chain.invoke({"input": page_content})

[{'title': 'Chain of thought (CoT)', 'author': 'Wei et al. 2022'},
 {'title': 'Tree of Thoughts', 'author': 'Yao et al. 2023'},
 {'title': 'LLM+P', 'author': 'Liu et al. 2023'},
 {'title': 'ReAct', 'author': 'Yao et al. 2023'},
 {'title': 'Reflexion', 'author': 'Shinn & Labash 2023'},
 {'title': 'Chain of Hindsight (CoH)', 'author': 'Liu et al. 2023'},
 {'title': 'Algorithm Distillation (AD)', 'author': 'Laskin et al. 2023'},
 {'title': 'RL^2', 'author': 'Duan et al. 2017'}]

In [ ]:
extraction_chain.invoke({"input": "hi"})

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_overlap=0)

splits = text_splitter.split_text(doc.page_content)

len(splits)

def flatten(matrix):
    flat_list = []
    for row in matrix:
        flat_list += row
    return flat_list

flatten([[1, 2], [3, 4]])


from langchain.schema.runnable import RunnableLambda

prep = RunnableLambda(
    lambda x: [{"input": doc} for doc in text_splitter.split_text(x)]
)

prep.invoke("hi")

chain = prep | extraction_chain.map() | flatten

chain.invoke(doc.page_content)



[{'title': 'Chain of thought', 'author': 'Wei et al. 2022'},
 {'title': 'Tree of Thoughts', 'author': 'Yao et al. 2023'},
 {'title': 'LLM+P', 'author': 'Liu et al. 2023'},
 {'title': 'ReAct', 'author': 'Yao et al. 2023'},
 {'title': 'Reflexion', 'author': 'Shinn & Labash 2023'},
 {'title': 'Chain of Hindsight', 'author': 'Liu et al.'},
 {'title': 'Algorithm Distillation', 'author': 'Laskin et al.'},
 {'title': None, 'author': 'Laskin et al. 2023'},
 {'title': None, 'author': 'Duan et al. 2017'},
 {'title': None, 'author': 'Miller 1956'},
 {'title': 'MRKL', 'author': 'Karpas et al. 2022'},
 {'title': 'TALM', 'author': 'Parisi et al. 2022'},
 {'title': 'Toolformer', 'author': 'Schick et al. 2023'},
 {'title': 'HuggingGPT', 'author': 'Shen et al. 2023'},
 {'title': 'API-Bank', 'author': 'Li et al. 2023'},
 {'title': 'ChemCrow', 'author': 'Bran et al. 2023'},
 {'title': None, 'author': 'Boiko et al. (2023)'},
 {'title': 'Generative Agents', 'author': 'Park, et al. 2023'},
 {'title': 'Chain

# Lesson 5 Tools and Routing

In [ ]:
from langchain.agents import tool

In [ ]:
@tool
def search(query:str)->str:
  """Search for weather online"""
  return "42f"

In [ ]:
search.name

'search'

In [ ]:
search.description

'search(query: str) -> str - Search for weather online'

In [ ]:
search.args

{'query': {'title': 'Query', 'type': 'string'}}

In [ ]:
from pydantic.v1 import BaseModel,Field
class SearchInput(BaseModel):
  query:str=Field(description="Thing to search for")

In [ ]:
@tool(args_schema=SearchInput)
def search(query: str)->str:
  """Search for weather online"""
  return "42f"

In [ ]:
search.args

{'query': {'title': 'Query', 'type': 'string'}}

In [ ]:
search.run('a')

'42f'

In [ ]:
import requests
from pydantic.v1 import BaseModel, Field
import datetime

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""

    BASE_URL = "https://api.open-meteo.com/v1/forecast"

    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)

    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']

    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]

    return f'The current temperature is {current_temperature}°C'

In [ ]:
get_current_temperature.run({"latitude":14,"longitude":13})

'The current temperature is 45.6°C'

In [ ]:
!pip install -q wikipedia

  Preparing metadata (setup.py) ... done


In [ ]:
import wikipedia
@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [ ]:
search_wikipedia({"query":"Wikipedia"})

"Page: Wikipedia\nSummary: Wikipedia is a free content online encyclopedia written and maintained by a community of volunteers, known as Wikipedians, through open collaboration and the use of the wiki-based editing system MediaWiki. Wikipedia is the largest and most-read reference work in history. It is consistently ranked as one of the ten most popular websites in the world, and as of 2024 is ranked the fifth most visited website on the Internet by Semrush, and second by Ahrefs. Founded by Jimmy Wales and Larry Sanger on January 15, 2001, Wikipedia is hosted by the Wikimedia Foundation, an American nonprofit organization that employs a staff of over 700 people.\nInitially only available in English, editions in other languages have been developed. Wikipedia's editions, when combined, comprise more than 62 million articles, attracting around 2 billion unique device visits per month and more than 14 million edits per month (about 5.2 edits per second on average)  as of November 2023. Rou

## Routing

In [ ]:
from langchain.tools.render import format_tool_to_openai_function
functions = [
    format_tool_to_openai_function(f) for f in [
        search_wikipedia, get_current_temperature
    ]
]
model = model.bind(functions=functions)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.2.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


In [ ]:
model.invoke("What is current temperature of Chennai")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "latitude": 13.0827,\n  "longitude": 80.2707\n}', 'name': 'get_current_temperature'}}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 128, 'total_tokens': 157}, 'model_name': 'gpt-4-32k', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'function_call', 'logprobs': None, 'content_filter_results': {}}, id='run-71fecd89-d1e8-4a3b-ab79-b2f3b1cf13fc-0')

In [ ]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])
chain = prompt | model

In [ ]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [ ]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [ ]:
result = chain.invoke({"input": "what is the weather in sf right now"})

**If openai does function call represented using AgentAction** \
**If openai doesn't function call represented using AgentFinish**

In [ ]:
type(result)

langchain_core.agents.AgentActionMessageLog

In [ ]:
result.tool

'get_current_temperature'

In [ ]:
result.tool_input

{'latitude': 37.7749, 'longitude': -122.4194}

In [ ]:
result.return_values

AttributeError: 'AgentActionMessageLog' object has no attribute 'return_values'

In [ ]:
result2 = chain.invoke({"input": "hi"})

In [ ]:
type(result2)

langchain_core.agents.AgentFinish

In [ ]:
result2.return_values

{'output': 'Hello there! How can I assist you today, preferably with a dash of sass?'}

In [ ]:
chain.invoke({"input":"Where is SF located? and what is current temp in SF?"})

AgentActionMessageLog(tool='search_wikipedia', tool_input={'query': 'San Francisco'}, log="\nInvoking: `search_wikipedia` with `{'query': 'San Francisco'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "query": "San Francisco"\n}', 'name': 'search_wikipedia'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 143, 'total_tokens': 160}, 'model_name': 'gpt-4-32k', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'function_call', 'logprobs': None, 'content_filter_results': {}}, id='run-3edf00a8-d03c-4cf4-a40f-929e328b8ad7-0')])

In [ ]:
from langchain.schema.agent import AgentFinish
def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "search_wikipedia": search_wikipedia,
            "get_current_temperature": get_current_temperature,
        }
        return tools[result.tool].run(result.tool_input)

In [ ]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route

In [ ]:
chain.invoke({"input": "What is the weather in san francisco right now?"})

'The current temperature is 11.9°C'

In [ ]:
chain.invoke({"input": "hi"})

'Hello there! How can I assist you today, with a sprinkle of sass?'

In [ ]:
chain.invoke({"input":"Where is SF located? and what is current temp in SF?"})

"San Francisco, often abbreviated as SF, is located in Northern California, USA. It's on the tip of a peninsula surrounded by the Pacific Ocean and San Francisco Bay. It's known for its hilly landscape, iconic Golden Gate Bridge, and year-round fog, among many other things.\n\nAs for the current temperature in San Francisco, unfortunately, I don't have real-time capabilities or access to live data feeds. To get the current temperature, you might want to check a reliable weather service or use a smart device that can provide you with real-time weather updates. Now, don't go out dressed for the wrong weather and blame it on me! Stay savvy and check that forecast."

## Testing parallel function call

In [ ]:
from langchain.tools.render import format_tool_to_openai_function
functions = [
    format_tool_to_openai_function(f) for f in [
        search_wikipedia, get_current_temperature
    ]]
model_with_parallel=model.bind_tools(tools=functions)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.2.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


In [ ]:
model_with_parallel.invoke({"input":"Where is SF located? and what is current temp in SF?"})

ValueError: Invalid input type <class 'dict'>. Must be a PromptValue, str, or list of BaseMessages.

In [ ]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant. you have access to following tools get_current_temperature and search_wikipedia. \
    You can use both tools in parallel to answer user query.\
    if user question has two sub questions you both tools to answer the query "),
    ("user", "{input}"),
])
chain_with_tools = prompt | model_with_parallel

In [ ]:
chain_with_tools.invoke({"input":"what is current temp in SF? and explain about SF"})

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_kiDRmgczpC4rt0CtNB57Sgde', 'function': {'arguments': '{"latitude": 37.7749, "longitude": -122.4194}', 'name': 'get_current_temperature'}, 'type': 'function'}, {'id': 'call_PACArOKPi6LejtfchPuJOHNV', 'function': {'arguments': '{"query": "San Francisco"}', 'name': 'search_wikipedia'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 93, 'prompt_tokens': 183, 'total_tokens': 276}, 'model_name': 'gpt-4', 'system_fingerprint': 'fp_2f57f81c11', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'tool_calls', 'logprobs': None, 'content_filter_results': {}}, id='run-9a4fefcf-4e83-4efd-8e40-e4bd25b6afa9-0', tool_calls=[{'name': 'get_current_temperature'

https://python.langchain.com/docs/use_cases/tool_use/parallel/

In [ ]:
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

In [ ]:
from langchain_core.tools import tool


@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int


@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int


@tool
def exponentiate(base: int, exponent: int) -> int:
    "Exponentiate the base to the exponent power."
    return base**exponent

In [ ]:
from operator import itemgetter
from typing import Dict, List, Union

from langchain_core.messages import AIMessage
from langchain_core.runnables import (
    Runnable,
    RunnableLambda,
    RunnableMap,
    RunnablePassthrough,
)

tools = [multiply, exponentiate, add]
llm_with_tools = model.bind_tools(tools)
tool_map = {tool.name: tool for tool in tools}


def call_tools(msg: AIMessage) -> Runnable:
    """Simple sequential tool calling helper."""
    tool_map = {tool.name: tool for tool in tools}
    tool_calls = msg.tool_calls.copy()
    for tool_call in tool_calls:
        tool_call["output"] = tool_map[tool_call["name"]].invoke(tool_call["args"])
    return tool_calls


chain = llm_with_tools | call_tools

In [ ]:
llm_with_tools.invoke("What's 23 times 7, and what's five times 18 and add a million plus a billion and cube thirty-seven")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_g81ORPtQOCfjgsBmHSrWn83v', 'function': {'arguments': '{"first_int": 23, "second_int": 7}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_Qd5hZ3Z6Qx2Kp4kc8EVPjlMf', 'function': {'arguments': '{"first_int": 5, "second_int": 18}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_1ZbbiIoeCxGvRJNMrPANfwk1', 'function': {'arguments': '{"first_int": 1000000, "second_int": 1000000000}', 'name': 'add'}, 'type': 'function'}, {'id': 'call_kUv4eic0QmxGoqiN7TMCGebY', 'function': {'arguments': '{"base": 37, "exponent": 3}', 'name': 'exponentiate'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 171, 'prompt_tokens': 168, 'total_tokens': 339}, 'model_name': 'gpt-4', 'system_fingerprint': 'fp_2f57f81c11', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual':

# Lesson 6 Conversational Agents

## Agents Basics

* Agents
  * are combination of LLM and code
  * LLM reason about what steps to take and call for actions

* Agent Loop
  * Choose a tool to use
  * Observe the output of tool
  * Repeat until stop condition is met
* Stopping conditions can be:
 * LLM determined
 *Hardcoded rules like max iterations



In [ ]:
from langchain.agents import tool

In [ ]:
import requests
from pydantic.v1 import BaseModel, Field
import datetime

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""

    BASE_URL = "https://api.open-meteo.com/v1/forecast"

    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)

    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']

    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]

    return f'The current temperature is {current_temperature}°C'

In [ ]:
import wikipedia
@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)